In [2]:
import pandas as pd

# Dataset Loading

In [20]:
train_bodies = pd.read_csv("dataset/train_bodies.csv")
train_stances = pd.read_csv("dataset/train_stances.csv")
train_dataset = train_stances.merge(train_bodies, on='Body ID', how="inner")

test_bodies = pd.read_csv("dataset/competition_test_bodies.csv")
test_stances = pd.read_csv("dataset/competition_test_stances.csv")
train_dataset = test_stances.merge(test_bodies, on='Body ID', how="inner")

                                                Headline  Body ID     Stance  \
0      Police find mass graves with at least '15 bodi...      712  unrelated   
1               Seth Rogen to Play Apple’s Steve Wozniak      712    discuss   
2      Mexico police find mass grave near site 43 stu...      712  unrelated   
3      Mexico Says Missing Students Not Found In Firs...      712  unrelated   
4      New iOS 8 bug can delete all of your iCloud do...      712  unrelated   
...                                                  ...      ...        ...   
49967  Amazon Is Opening a Brick-and-Mortar Store in ...      464      agree   
49968  Elon University has not banned the term ‘fresh...      362      agree   
49969  Fake BBC News website set up to carry Charlie ...      915      agree   
49970  Apple was forced to nix key health features fr...      407    discuss   
49971  If you want an Apple Watch, you’ll probably be...     1066    discuss   

                                       

# Feature Extraction